In [82]:
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset


In [76]:
x_train = np.load('./data/colored_EMNIST/x_train_colored.npy')
y_train_colors = np.load('./data/colored_EMNIST/y_train_colors.npy')
x_test = np.load('./data/colored_EMNIST/x_test_colored.npy')
y_test_colors = np.load('./data/colored_EMNIST/y_test_colors.npy')

In [100]:
x_train_combined = np.load('./data/colored_EMNIST/x_train_colored.npy')
y_train_combined = np.load('./data/colored_EMNIST/y_train_combined.npy')
x_test_combined = np.load('./data/colored_EMNIST/x_test_colored.npy')
y_test_combined = np.load('./data/colored_EMNIST/y_test_combined.npy')

In [78]:
print(x_train.shape)
print(y_train_colors.shape)
print(x_test.shape)
print(y_test_colors.shape)

(240000, 28, 28, 3)
(240000,)
(40000, 28, 28, 3)
(40000,)


In [101]:
print(x_train_combined.shape)
print(y_train_combined.shape)
print(x_test_combined.shape)
print(y_test_combined.shape)

(240000, 28, 28, 3)
(240000,)
(40000, 28, 28, 3)
(40000,)


In [105]:
# unique x_train_combined
print(np.unique(y_train_combined))

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95
 96 97 98 99]


In [83]:
# Convert numpy arrays to torch tensors
x_train_tensor = torch.tensor(x_train, dtype=torch.float32).permute(0, 3, 1, 2) / 255.0
y_train_tensor = torch.tensor(y_train_colors, dtype=torch.long)
x_test_tensor = torch.tensor(x_test, dtype=torch.float32).permute(0, 3, 1, 2) / 255.0
y_test_tensor = torch.tensor(y_test_colors, dtype=torch.long)

train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
test_dataset = TensorDataset(x_test_tensor, y_test_tensor)
num_classes = 10

In [98]:
unique, counts = np.unique(y_train_colors, return_counts=True)
print(dict(zip(unique, counts)))

{0: 24000, 1: 24000, 2: 24000, 3: 24000, 4: 24000, 5: 24000, 6: 24000, 7: 24000, 8: 24000, 9: 24000}


In [110]:
labels = train_dataset.tensors[1]
print(labels)

tensor([0, 0, 0,  ..., 9, 9, 9])


In [99]:
# train model with resnet18
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.models as models

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

#make label noise
train_loader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = models.resnet18(pretrained=False)
model.fc = nn.Linear(512, num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print(f'Epoch {epoch + 1}/{num_epochs}, Step {i + 1}/{len(train_loader)}, Loss: {loss.item():.4f}')

    model.eval()
    with torch.no_grad():
        correct, total = 0, 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        accuracy = correct / total * 100
        print(f'Epoch {epoch + 1}/{num_epochs}, Accuracy: {accuracy:.2f}%')

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch 1/10, Step 100/3750, Loss: 0.0135
Epoch 1/10, Step 200/3750, Loss: 0.0016
Epoch 1/10, Step 300/3750, Loss: 0.0002
Epoch 1/10, Step 400/3750, Loss: 0.1517
Epoch 1/10, Step 500/3750, Loss: 0.0032
Epoch 1/10, Step 600/3750, Loss: 0.0004
Epoch 1/10, Step 700/3750, Loss: 0.0054
Epoch 1/10, Step 800/3750, Loss: 0.0001
Epoch 1/10, Step 900/3750, Loss: 0.0001
Epoch 1/10, Step 1000/3750, Loss: 0.0036
Epoch 1/10, Step 1100/3750, Loss: 1.2310
Epoch 1/10, Step 1200/3750, Loss: 0.0077
Epoch 1/10, Step 1300/3750, Loss: 0.0076
Epoch 1/10, Step 1400/3750, Loss: 0.0010
Epoch 1/10, Step 1500/3750, Loss: 0.0002
Epoch 1/10, Step 1600/3750, Loss: 0.0001
Epoch 1/10, Step 1700/3750, Loss: 0.0010
Epoch 1/10, Step 1800/3750, Loss: 0.0003
Epoch 1/10, Step 1900/3750, Loss: 0.0001
Epoch 1/10, Step 2000/3750, Loss: 0.0000
Epoch 1/10, Step 2100/3750, Loss: 0.0002
Epoch 1/10, Step 2200/3750, Loss: 0.0000
Epoch 1/10, Step 2300/3750, Loss: 0.0003
Epoch 1/10, Step 2400/3750, Loss: 0.0001
Epoch 1/10, Step 2500/375